# Nạp các file đầu vào

In [ ]:
# INPUT file  : input.xlsx, sheet "input"
# REF   file  :
# Thư viện: pip install pyodbc pandas openpyxl
# Thư viện: pip install sqlalchemy pyodbc

In [ ]:
import subprocess
import sys
libraries = ['pyodbc', 'sqlalchemy', 'openpyxl', 'xlsxwriter', 'pandas', 'urllib', 'math', 'os', 'datetime', 'scikit-learn', 'numpy', 'sys', 'unidecode']

for lib in libraries:
    try:
        __import__(lib)
        print(f"{lib} : done")
    except ImportError:
        print(f"{lib} : not yet >> Installing...")
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', lib])
        print(f"{lib} : done")

In [ ]:
# nạp các thư viện
# import sys
# sys.path.append('c:/Users/Phanl/Desktop/tf_idf_with_cosine_similarity')

import pandas as pd
from sqlalchemy import create_engine
import urllib
import sql_user.sql_user_login as login
from functions import query_exportExcel, distance_km, TFIDF_CosineSimilarity, unicode_convert_to_ascii, remove_special_characters, preprocess_text
from datetime import datetime
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import os
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Alignment, NamedStyle

In [ ]:
# nạp file input
query_exportExcel(sql_file_path = 'sql_script/input_query.sql', output_file = 'input/input.xlsx', sheet_name = 'input')
df_input = pd.read_excel('input/input.xlsx', sheet_name='input', usecols=['ROW_NO', 'CUSTOMER_NAME', 'CUSTOMER_LAT', 'CUSTOMER_LONG', 'CUSTOMER_ADDRESS'])
print(df_input.head(10))

In [ ]:
# Phải convert chúng về non-unicode và gỡ bỏ toàn bộ ký tự đặc biệt mới tính độ tương đồng được
# df_input['CUSTOMER_NAME'] = df_input['CUSTOMER_NAME'].apply(unicode_convert_to_ascii)
# df_input['CUSTOMER_NAME'] = df_input['CUSTOMER_NAME'].apply(remove_special_characters)
df_input['SEARCH_NAME_FIXED']      = df_input['CUSTOMER_NAME'].apply(preprocess_text)
df_input['SEARCH_ADDRESS_FIXED']   = df_input['CUSTOMER_ADDRESS'].apply(preprocess_text)

df_input['SEARCH_COMBINE_FIXED']   = df_input['CUSTOMER_NAME'] + " " + df_input['CUSTOMER_ADDRESS']
df_input['SEARCH_COMBINE_FIXED']   = df_input['SEARCH_COMBINE_FIXED'].apply(preprocess_text)

# Thêm các cột mới vào df_input
df_input['MATCH_CUSTOMER_CODE']    = None
df_input['MATCH_CUSTOMER_NAME']    = None
df_input['MATCH_CUSTOMER_ADDRESS'] = None
df_input['MATCH_DISTANCE_KM']      = None

df_input['MATCH_NAME_FIXED']       = None
df_input['MATCH_ADDRESS_FIXED']    = None
df_input['MATCH_COMBINE_FIXED']    = None
df_input['MATCH_NAME_SCORE']       = None
df_input['MATCH_ADDRESS_SCORE']    = None
df_input['MATCH_COMBINE_SCORE']    = None
df_input['MATCH_FINAL_SCORE']      = None

df_input['CLOSEST_DISTANCE_KM']    = None

print(df_input.head())

In [ ]:
# nạp file ref
# nên comment đoạn query lại, vì query hơi lâu
query_exportExcel(sql_file_path = 'sql_script/ref_query.sql', output_file = 'input/ref.xlsx', sheet_name = 'ref')
df_ref = pd.read_excel('input/ref.xlsx', sheet_name='ref', usecols=['CUSTOMER_CODE', 'CUSTOMER_NAME', 'CUSTOMER_LAT', 'CUSTOMER_LONG','CUSTOMER_ADDRESS'])
print(df_ref.head(10))

In [ ]:
# Phải convert chúng về non-unicode và gỡ bỏ toàn bộ ký tự đặc biệt mới tính độ tương đồng được
# df_ref ['CUSTOMER_NAME'] = df_ref ['CUSTOMER_NAME'].apply(unicode_convert_to_ascii)
# df_ref ['CUSTOMER_NAME'] = df_ref ['CUSTOMER_NAME'].apply(remove_special_characters)
df_ref ['SEARCH_NAME_FIXED']      = df_ref ['CUSTOMER_NAME'].apply(preprocess_text)
df_ref ['SEARCH_ADDRESS_FIXED']   = df_ref ['CUSTOMER_ADDRESS'].apply(preprocess_text)

df_ref ['SEARCH_COMBINE_FIXED']   = df_ref ['CUSTOMER_NAME'] + " " + df_ref ['CUSTOMER_ADDRESS']
df_ref ['SEARCH_COMBINE_FIXED']   = df_ref ['SEARCH_COMBINE_FIXED'].apply(preprocess_text)
print(df_ref .head())

In [ ]:
# Xử lý từng record trong df_input
distance_standard = 0.05 #50m

# for i, input_row in df_input.head(10).iterrows():
for i, input_row in df_input.iterrows():    
    # Copy dữ liệu từ df_ref vào df_ref_temp
    df_ref_temp = df_ref.copy()

# Khả năng là các danh sách loại trừ sẽ khiến cho các match theo name bị hạn chế --> mở ràng
    ## Lấy danh sách CUSTOMER_CODE đã tồn tại trong df_input['MATCH_CUSTOMER_CODE']
    #existing_codes = df_input['MATCH_CUSTOMER_CODE'].dropna().unique()
    ## Xóa các bản ghi trong df_ref_temp mà CUSTOMER_CODE đã tồn tại
    #df_ref_temp = df_ref_temp[~df_ref_temp['CUSTOMER_CODE'].isin(existing_codes)]    
    
    # Thêm cột DISTANCE_KM vào df_ref_temp và tính khoảng cách
    df_ref_temp['MATCH_DISTANCE_KM'] = df_ref_temp.apply(
        lambda row: distance_km(
            input_row['CUSTOMER_LAT'], input_row['CUSTOMER_LONG'],
            row['CUSTOMER_LAT'], row['CUSTOMER_LONG']
        ),
        axis=1
    )
    
    # Tìm Closest_distance_km
    closest_distance_km = df_ref_temp['MATCH_DISTANCE_KM'].min()

    # Lọc những record có khoảng cách <= 0.05 và sắp xếp theo DISTANCE_KM giảm dần
    df_ref_temp = df_ref_temp[df_ref_temp['MATCH_DISTANCE_KM'] <= distance_standard]
    
    # Kiểm tra nếu df_ref_temp rỗng
    if df_ref_temp.empty:
        # Nếu rỗng, gán giá trị "not match distance" và để các cột còn lại trống
        df_input.at[i, 'MATCH_CUSTOMER_CODE']    = f'NOT MATCH DISTANCE: {distance_standard * 1000} m'
        df_input.at[i, 'MATCH_CUSTOMER_NAME']    = None
        df_input.at[i, 'MATCH_CUSTOMER_ADDRESS'] = None
        df_input.at[i, 'MATCH_DISTANCE_KM']      = None
        df_input.at[i, 'MATCH_NAME_FIXED']       = None
        df_input.at[i, 'MATCH_ADDRESS_FIXED']    = None
        df_input.at[i, 'MATCH_COMBINE_FIXED']    = None
        df_input.at[i, 'MATCH_NAME_SCORE']       = None
        df_input.at[i, 'MATCH_ADDRESS_SCORE']    = None
        df_input.at[i, 'MATCH_COMBINE_SCORE']    = None
        df_input.at[i, 'MATCH_FINAL_SCORE']      = None
        df_input.at[i, 'CLOSEST_DISTANCE_KM']    = closest_distance_km

    else:
        # Nếu không rỗng, tiếp tục xử lý
        # df_ref_temp = df_ref_temp.sort_values(by='DISTANCE_KM', ascending=False)
        
        # Thêm cột MATCH_NAME_SCORE vào df_ref_temp và tính độ giống nhau về tên
        ref_names_fixed = df_ref_temp['SEARCH_NAME_FIXED'].tolist()  # Chuyển đổi cột thành list
        df_ref_temp['MATCH_NAME_SCORE'] = df_ref_temp.apply(
            lambda row: TFIDF_CosineSimilarity(
                input_row['SEARCH_NAME_FIXED'], ref_names_fixed
            ),
            axis=1
        )

        # Thêm cột MATCH_ADDRESS_SCORE vào df_ref_temp và tính độ giống nhau về địa chỉ
        ref_address_fixed = df_ref_temp['SEARCH_ADDRESS_FIXED'].tolist()  # Chuyển đổi cột thành list
        df_ref_temp['MATCH_ADDRESS_SCORE'] = df_ref_temp.apply(
            lambda row: TFIDF_CosineSimilarity(
                input_row['SEARCH_ADDRESS_FIXED'], ref_address_fixed
            ),
            axis=1
        )

        # Thêm cột MATCH_COMBINE_SCORE vào df_ref_temp và tính độ giống nhau về [tên + địa chỉ]
        ref_combine_fixed = df_ref_temp['SEARCH_COMBINE_FIXED'].tolist()  # Chuyển đổi cột thành list
        df_ref_temp['MATCH_COMBINE_SCORE'] = df_ref_temp.apply(
            lambda row: TFIDF_CosineSimilarity(
                input_row['SEARCH_COMBINE_FIXED'], ref_combine_fixed
            ),
            axis=1
        )  
        
        # Tổng hợp điểm số Match theo bộ 3: name, address, [name + address]
        df_ref_temp['MATCH_FINAL_SCORE'] = df_ref_temp['MATCH_NAME_SCORE'].fillna(1) * df_ref_temp['MATCH_ADDRESS_SCORE'].fillna(1) * df_ref_temp['MATCH_COMBINE_SCORE'].fillna(1)


        # Loại bỏ một số records khi:
        # + không match Customer name nào trong bán kính ban đầu (50m)
        df_ref_temp = df_ref_temp[df_ref_temp['MATCH_NAME_SCORE'] != 0]

        # Kiểm tra thêm 1 lần nữa, nếu df_ref_temp rỗng
        if df_ref_temp.empty:
            # Nếu rỗng, gán giá trị "not match distance" và để các cột còn lại trống
            df_input.at[i, 'MATCH_CUSTOMER_CODE']    = f'NOT MATCH NAME'
            df_input.at[i, 'MATCH_CUSTOMER_NAME']    = None
            df_input.at[i, 'MATCH_CUSTOMER_ADDRESS'] = None
            df_input.at[i, 'MATCH_DISTANCE_KM']      = None
            df_input.at[i, 'MATCH_NAME_FIXED']       = None
            df_input.at[i, 'MATCH_ADDRESS_FIXED']    = None
            df_input.at[i, 'MATCH_COMBINE_FIXED']    = None
            df_input.at[i, 'MATCH_NAME_SCORE']       = None
            df_input.at[i, 'MATCH_ADDRESS_SCORE']    = None
            df_input.at[i, 'MATCH_COMBINE_SCORE']    = None
            df_input.at[i, 'MATCH_FINAL_SCORE']      = None
            df_input.at[i, 'CLOSEST_DISTANCE_KM']    = closest_distance_km

        else:
            # Sắp xếp theo MATCH_FINAL_SCORE giảm dần
            df_ref_temp = df_ref_temp.sort_values(by='MATCH_FINAL_SCORE', ascending=False)

            # Chỉ giữ lại record đầu tiên
            best_match = df_ref_temp.iloc[0]

            # Ghi kết quả search được vào bảng gốc
            df_input.at[i, 'MATCH_CUSTOMER_CODE']    = best_match['CUSTOMER_CODE']
            df_input.at[i, 'MATCH_CUSTOMER_NAME']    = best_match['CUSTOMER_NAME']
            df_input.at[i, 'MATCH_CUSTOMER_ADDRESS'] = best_match['CUSTOMER_ADDRESS']
            df_input.at[i, 'MATCH_DISTANCE_KM']      = best_match['MATCH_DISTANCE_KM']
            df_input.at[i, 'MATCH_NAME_FIXED']       = best_match['SEARCH_NAME_FIXED']
            df_input.at[i, 'MATCH_ADDRESS_FIXED']    = best_match['SEARCH_ADDRESS_FIXED']
            df_input.at[i, 'MATCH_COMBINE_FIXED']    = best_match['SEARCH_COMBINE_FIXED']
            df_input.at[i, 'MATCH_NAME_SCORE']       = best_match['MATCH_NAME_SCORE']
            df_input.at[i, 'MATCH_ADDRESS_SCORE']    = best_match['MATCH_ADDRESS_SCORE']
            df_input.at[i, 'MATCH_COMBINE_SCORE']    = best_match['MATCH_COMBINE_SCORE']
            df_input.at[i, 'MATCH_FINAL_SCORE']      = best_match['MATCH_FINAL_SCORE']
            df_input.at[i, 'CLOSEST_DISTANCE_KM']    = closest_distance_km

    # Xóa bảng df_ref_temp để qua vòng lặp mới
    del df_ref_temp

In [ ]:
# print 10 dòng kết quả trước khi lưu file
# print(df_input.head(2))

In [ ]:
# Tạo folder Output
if not os.path.exists('output'):
    os.makedirs('output')

In [ ]:
# Ghi kết quả vào file Excel
# Lựa chọn các cột cần xuất ra
columns_to_export = [
    'ROW_NO', 
    'CUSTOMER_NAME', 
    'CUSTOMER_LAT', 
    'CUSTOMER_LONG',
    'CUSTOMER_ADDRESS', 
    'MATCH_CUSTOMER_CODE', 
    'MATCH_CUSTOMER_NAME',
    'MATCH_CUSTOMER_ADDRESS', 
    'MATCH_DISTANCE_KM', 
    'MATCH_NAME_SCORE',
    'MATCH_ADDRESS_SCORE', 
    'MATCH_COMBINE_SCORE', 
    'MATCH_FINAL_SCORE',
    'CLOSEST_DISTANCE_KM'
]
df_output = df_input[columns_to_export]

# Sắp xếp data theo ROW_NO tăng dần
df_output = df_output.sort_values(by=['ROW_NO'])

# Ghi kết quả vào file Excel
output_file = f'output/output_{datetime.now().strftime("%Y%m%d_%H%M%S")}.xlsx'
df_output.to_excel(output_file, sheet_name='output', index=False)
print(f"Kết quả đã được ghi vào {output_file}")